In [7]:
# 1-) Yeni Ürünlerin sektörlerini bulmak

import pandas as pd

# CSV dosyalarını yükle
df_2007 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS07_V202401\product_codes_HS07_V202401.csv")
df_2012 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS12_V202401\product_codes_HS12_V202401.csv")
df_2017 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS17_V202401\product_codes_HS17_V202401.csv")
df_2022 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS22_V202401\product_codes_HS22_V202401.csv")

# 2007, 2012, 2017 ve 2022 yıllarına ait product code'ları set yap
products_2007 = set(df_2007['code'])
products_2012 = set(df_2012['code'])
products_2017 = set(df_2017['code'])
products_2022 = set(df_2022['code'])

# 2012, 2017 ve 2022'de yeni eklenen ürünler
new_products_2012 = products_2012 - products_2007
new_products_2012_df = df_2012[df_2012['code'].isin(new_products_2012)]
new_products_2017 = products_2017 - products_2012
new_products_2017_df = df_2017[df_2017['code'].isin(new_products_2017)]
new_products_2022 = products_2022 - products_2017
new_products_2022_df = df_2022[df_2022['code'].isin(new_products_2022)]

# Üç yıldaki yeni ürünleri aynı data frame içine al
combined_new_products = pd.concat([new_products_2022_df, new_products_2017_df, new_products_2012_df], 
                                  keys=['2022', '2017', '2012'], names=['Yıl', 'Index'])

# 6 basamaklı HS kodlarını 10000'e bölerek 2 basamaklı sektörlerini bul
combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['code'] // 10000

# Atlas verisini çağırıp 2 basamaklı HS kodlarının isimlerini çek
combined_atlas_df = pd.read_excel('0-) Atlastan Çekilmiş HS92 İsimleri 2 Basamak İçin.xlsx')
combined_atlas_df['Code'] = combined_atlas_df['Class_2_Data'].str.extract(r'\((\d{2})')

# Bir sözlük oluşturarak 2 basamaklı HS kodları ve isimleri arasında bir map oluştur
code_to_name_mapping = dict(zip(combined_atlas_df['Code'], combined_atlas_df['Class_1_Data']))
combined_new_products['Sektör İsmi'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2).map(code_to_name_mapping)

# Sütunları ismini değiştir
combined_new_products = combined_new_products.rename(columns={'description': 'Detaylı Sektör İsmi', 'code': '6 Basamaklı HS Kodu'})  

# Excele kaydet
combined_new_products.to_excel("1-) 2012, 2017 ve 2022'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx")

In [ ]:
# 2-) Yeni ürünlerin sektörlerdeki sayıları

import pandas as pd

# Dosyayı yükle
new_products_df = pd.read_excel('1-) 2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')

# Yıl kolonundaki boşlukları doldur
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# Her yıl için sektör isimlerine ve 2 basamaklı HS koduna göre grupla
sector_grouping_by_year = new_products_df.groupby(['Yıl', 'Sektör İsmi', '2 Basamaklı HS Kodu']).size().reset_index(name='Sayı')

# Sonuçları Excel dosyasına kaydet
sector_grouping_by_year.to_excel('2-) 2012, 2017 ve 2022\'deki yeni ürünlerin sektörlerdeki sayıları.xlsx', index=False)

In [11]:
# 3-) Yeni ürünlerin ticaret hacmindeki toplam oranları 2012-2021

import pandas as pd

# Yıllar ve ilgili HS sürümleri 
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı 
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('1-) 2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]

# Sonuçları tutmak için bir liste
toplam_oranlar_list = []

# Her yıl için döngü
for year in yillar:
    hs_version = hs_versiyonlari[year] 
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)  
    
    # Veriyi okuma
    df = pd.read_csv(dosya_yolu)
    
    # Toplam ticaret hacmini hesapla
    toplam_ticaret_hacmi = df['v'].sum()

    # 2012-2016 için 2012'deki yeni ürünler, 2017-2021 için 2017'deki yeni ürünler kullan
    if year <= 2016:
        yeni_urunler_ticaret_hacmi = df[df['k'].isin(new_products_2012_df['6 Basamaklı HS Kodu'])]['v'].sum()
    else:
        yeni_urunler_ticaret_hacmi = df[df['k'].isin(new_products_2017_df['6 Basamaklı HS Kodu'])]['v'].sum()
    
    # Yeni ürünlerin ticaret hacminin toplam ticaret hacmine oranı
    yeni_urunler_orani = (yeni_urunler_ticaret_hacmi / toplam_ticaret_hacmi) * 100
    
    # Sonuçları listeye ekle
    toplam_oranlar_list.append({'Yıl': year, 'Yeni Ürünlerin Oranı (%)': yeni_urunler_orani})

# Sonuçları DataFrame'e çevir ve excel olarak kaydet
toplam_oranlar = pd.DataFrame(toplam_oranlar_list)

toplam_oranlar.to_excel("3-) Yeni ürünlerin ticaret hacmindeki toplam oranları 2012-2021.xlsx", index=False)

In [5]:
# Yeni ürünlerin ticaret hacmindeki sektörel bazlı oranları 2012-2021

import pandas as pd

# Yıllar ve ilgili HS sürümleri 
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı 
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]



,Yıl,6 Basamaklı HS Kodu,Detaylı Sektör İsmi,2 Basamaklı HS Kodu,Sektör İsmi
0,2022.0,30910,"Flours, meals and pellets of fish, fit for hum...",3,Fish (03 HS92)
1,2022.0,30990,"Flours, meals and pellets of crustaceans, moll...",3,Fish (03 HS92)
2,2022.0,40320,"Dairy produce: yoghurt, whether or not concent...",4,Diary products (04 HS92)
3,2022.0,41010,Animal products: insects,4,Diary products (04 HS92)
4,2022.0,41090,Animal products: edible products of animal ori...,4,Diary products (04 HS92)
...,...,...,...,...,...
955,2012.0,930110,Military weapons: artillery weapons (e.g. guns...,93,Arms and ammunition (93 HS92)
956,2012.0,930520,"Firearms: parts and accessories, of shotguns o...",93,Arms and ammunition (93 HS92)
957,2012.0,950450,"Games: video game consoles and machines, other...",95,Toys (95 HS92)
958,2012.0,960830,"Pens: fountain, stylograph and other pens",96,Miscellaneous manufactured articles (96 HS92)


In [6]:
#Deneme ve düzeltme
import pandas as pd

# Yıllar ve ilgili HS sürümleri 
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı 
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]

# Ülkeler dosyasını yükle
ulkeler_df = pd.read_excel('Ülkeler.xlsx')

# Sonuçları tutmak için bir DataFrame oluştur
sonuclar_sektorel = pd.DataFrame()

# Her ülke ve her yıl için işlem yapma
for index, row in ulkeler_df.iterrows():
    ulke = row['Countries']  
    ulke_kodu = row['country_code']  
    
    for year in yillar:
        hs_version = hs_versiyonlari[year] 
        dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)
        
        # Veriyi yükle
        df = pd.read_csv(dosya_yolu)
        
        # Yalnızca ilgili ülkeye ait veriyi filtreleme
        ulke_veri = df[df['i'] == ulke_kodu]
        
        # Toplam ticaret hacmini sektör bazlı hesapla
        ulke_veri_sektorel = ulke_veri.groupby('k')['v'].sum().reset_index()
        
        # 2012-2016 için 2012 ürünleri, 2017-2021 için 2017 ürünleri kullanılıyor
        if year <= 2016:
            yeni_urunler_sektorel = ulke_veri[ulke_veri['k'].isin(new_products_2012_df['6 Basamaklı HS Kodu'])]
        else:
            yeni_urunler_sektorel = ulke_veri[ulke_veri['k'].isin(new_products_2017_df['6 Basamaklı HS Kodu'])]
        
        # Yeni ürünlerin ticaret hacmini sektör bazlı hesapla
        yeni_urunler_sektorel_hacim = yeni_urunler_sektorel.groupby('k')['v'].sum().reset_index()

        # Yeni ürünlerin ticaret hacminin toplam ticaret hacmine oranını sektör bazlı hesaplama
        for sektor in yeni_urunler_sektorel_hacim['k'].unique():
            toplam_ticaret_hacmi = ulke_veri_sektorel[ulke_veri_sektorel['k'] == sektor]['v'].sum()
            yeni_urun_ticaret_hacmi = yeni_urunler_sektorel_hacim[yeni_urunler_sektorel_hacim['k'] == sektor]['v'].sum()
            
            if toplam_ticaret_hacmi > 0:
                oran = (yeni_urun_ticaret_hacmi / toplam_ticaret_hacmi) * 100
            else:
                oran = 0
            
            # Sonuçları kaydet
            sonuclar_sektorel.at[f"{ulke}_{sektor}", year] = oran

# Sonuçları Excel dosyasına kaydet
sonuclar_sektorel.to_excel('yeni_urunler_sektorel_oran_CEE_MENA.xlsx')

# Sonuçları göster
print(sonuclar_sektorel)


                    2012   2013   2014   2015   2016  2017  2018  2019  2020  \
Albania_20910      100.0    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
Albania_30213      100.0    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
Albania_30243      100.0  100.0  100.0  100.0  100.0   NaN   NaN   NaN   NaN   
Albania_30244      100.0  100.0  100.0  100.0  100.0   NaN   NaN   NaN   NaN   
Albania_30251      100.0  100.0  100.0    NaN    NaN   NaN   NaN   NaN   NaN   
...                  ...    ...    ...    ...    ...   ...   ...   ...   ...   
Bangladesh_370500    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
Bangladesh_401170    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
Bangladesh_441875    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
Bangladesh_842441    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
Bangladesh_845612    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   

                    2021  
Albania_2091

In [7]:
# CEE/MENA bölgesinde ticaret hacminde yeni ürünlerin toplam oranı 2012-2021

import pandas as pd

# Yıllar ve ilgili HS sürümleri
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı 
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]

# Ülkeler dosyasını yükle
ulkeler_df = pd.read_excel('Ülkeler.xlsx')

# Sonuçları tutmak için bir DataFrame oluştur
sonuclar3 = pd.DataFrame()

# Her ülke ve her yıl için işlem yapma
for index, row in ulkeler_df.iterrows():
    ulke = row['Countries']  
    ulke_kodu = row['country_code']  
    for year in yillar:
        hs_version = hs_versiyonlari[year] 
        dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)
        
        # Veriyi yükle
        df = pd.read_csv(dosya_yolu)
        
        # Yalnızca ilgili ülkeye ait veriyi filtreleme
        ulke_veri = df[df['i'] == ulke_kodu]
        toplam_ticaret_hacmi = ulke_veri['v'].sum()
        
        # 2012-2016 için 2012 ürünleri, 2017-2021 için 2017 ürünleri kullanılıyor
        if year <= 2016:
            yeni_urunler_ticaret_hacmi = ulke_veri[ulke_veri['k'].isin(new_products_2012_df['6 Basamaklı HS Kodu'])]['v'].sum()
        else:
            yeni_urunler_ticaret_hacmi = ulke_veri[ulke_veri['k'].isin(new_products_2017_df['6 Basamaklı HS Kodu'])]['v'].sum()

        # Yeni ürünlerin ticaret hacminin toplam ticaret hacmine oranı
        if toplam_ticaret_hacmi > 0:  
            yeni_urunler_orani = (yeni_urunler_ticaret_hacmi / toplam_ticaret_hacmi) * 100
        else:
            yeni_urunler_orani = 0

        # Sonuçları DataFrame'e kaydet
        sonuclar3.at[ulke, year] = yeni_urunler_orani

# Sonuçları göster
print(sonuclar3)

# Sonuçları Excel dosyasına kaydet


                               2012      2013          2014      2015  \
Albania                    0.000000  0.023142  0.000000e+00  0.000000   
Bosnia Herzegovina         0.000000  0.000000  0.000000e+00  0.000000   
Bulgaria                   0.000310  0.000017  1.616380e-04  0.000341   
Croatia                    0.000133  0.000042  1.281598e-04  0.000155   
Czech Republic             0.000475  0.000962  1.166677e-03  0.001483   
Estonia                    0.000161  0.001095  6.178623e-04  0.001328   
Hungary                    0.000379  0.000122  3.386638e-04  0.000519   
Latvia                     0.000181  0.000180  7.101429e-06  0.000713   
Lithuania                  0.000794  0.000316  1.687069e-03  0.001423   
Montenegro                 0.000000  0.000000  0.000000e+00  0.000000   
North Macedonia            0.000000  0.000000  0.000000e+00  0.000022   
Poland                     0.001130  0.001722  1.246067e-03  0.001688   
Romania                    0.000000  0.000000  6.70

In [3]:
# 2012'de ve 2017'de yeni eklenen ürünlerin CEE/MENA bölgesinde ticaret hacminde oranları 2012-2021

import pandas as pd

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]

# Yıllar ve ilgili HS sürümleri
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"
ulkeler_df = pd.read_excel('Ülkeler.xlsx')

sonuclar3 = pd.DataFrame()
for index, row in ulkeler_df.iterrows():
    ulke = row['Countries']  
    ulke_kodu = row['country_code']  
    for year in yillar:
        hs_version = hs_versiyonlari[year] 
        dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)
        df = pd.read_csv(dosya_yolu)
        
        # Yalnızca ilgili ülkeye ait veriyi filtreleme
        ulke_veri = df[df['i'] == ulke_kodu]  
        toplam_ticaret_hacmi = ulke_veri['v'].sum()
        
        # 2012-2016 için new_products_2012_df, 2017-2021 için new_products_2017_df 
        if year <= 2016:
            yeni_urunler_ticaret_hacmi = ulke_veri[ulke_veri['k'].isin(new_products_2012_df['6 Basamaklı HS Kodu'])]['v'].sum()
        else:
            yeni_urunler_ticaret_hacmi = ulke_veri[ulke_veri['k'].isin(new_products_2017_df['6 Basamaklı HS Kodu'])]['v'].sum()

        if toplam_ticaret_hacmi > 0:  
            yeni_urunler_orani = (yeni_urunler_ticaret_hacmi / toplam_ticaret_hacmi) * 100
        else:
            yeni_urunler_orani = 0
        sonuclar3.at[ulke, year] = yeni_urunler_orani

# Sonuçları göster
print(sonuclar3)

# Sonuçları Excel'e kaydetmek isterseniz
sonuclar3


                               2012      2013          2014      2015  \
Albania                    0.000000  0.023142  0.000000e+00  0.000000   
Bosnia Herzegovina         0.000000  0.000000  0.000000e+00  0.000000   
Bulgaria                   0.000310  0.000017  1.616380e-04  0.000341   
Croatia                    0.000133  0.000042  1.281598e-04  0.000155   
Czech Republic             0.000475  0.000962  1.166677e-03  0.001483   
Estonia                    0.000161  0.001095  6.178623e-04  0.001328   
Hungary                    0.000379  0.000122  3.386638e-04  0.000519   
Latvia                     0.000181  0.000180  7.101429e-06  0.000713   
Lithuania                  0.000794  0.000316  1.687069e-03  0.001423   
Montenegro                 0.000000  0.000000  0.000000e+00  0.000000   
North Macedonia            0.000000  0.000000  0.000000e+00  0.000022   
Poland                     0.001130  0.001722  1.246067e-03  0.001688   
Romania                    0.000000  0.000000  6.70

,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Albania,0.000000,0.023142,0.000000e+00,0.000000,3.870910e-05,0.000000e+00,8.572043e-05,0.000000e+00,0.000000e+00,0.000000e+00
Bosnia Herzegovina,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Bulgaria,0.000310,0.000017,1.616380e-04,0.000341,1.336387e-06,7.283455e-06,5.310467e-05,2.617817e-05,3.882530e-05,1.645479e-05
Croatia,0.000133,0.000042,1.281598e-04,0.000155,5.770379e-05,1.896621e-02,9.680428e-03,1.168071e-03,1.018568e-03,4.862499e-04
Czech Republic,0.000475,0.000962,1.166677e-03,0.001483,3.453616e-04,4.021730e-07,1.136923e-06,5.125088e-10,1.587554e-07,6.487477e-07
Estonia,0.000161,0.001095,6.178623e-04,0.001328,7.283763e-04,1.136702e-04,5.654863e-05,1.461306e-05,6.462998e-05,7.649239e-05
Hungary,0.000379,0.000122,3.386638e-04,0.000519,1.034535e-04,8.757881e-09,5.119018e-06,1.146807e-06,2.929839e-06,1.936656e-05
Latvia,0.000181,0.000180,7.101429e-06,0.000713,1.744660e-04,3.607082e-05,1.765112e-04,1.659109e-05,2.917505e-05,1.264909e-04
Lithuania,0.000794,0.000316,1.687069e-03,0.001423,3.882190e-04,1.771083e-06,1.376568e-06,0.000000e+00,8.360388e-05,3.903670e-06
Montenegro,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,6.219265e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
# Ülkelerin ticaret hacminde yeni ürünlerin sektörel oranı 2012-2021


In [ ]:
# RCA örnek